In [ ]:
%pip install --upgrade git+https://github.com/huggingface/transformers.git 
!sudo apt-get update
!sudo apt-get install -y poppler-utils


In [ ]:
%pip install byaldi  accelerate  qwen_vl_utils pdf2image

In [ ]:
from byaldi import RAGMultiModalModel
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
from pdf2image import convert_from_path

RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")


In [ ]:
RAG.index(
    input_path="/home/dataset1.pdf",
    index_name="index",
    store_collection_with_index=False,
    overwrite=True
)

In [5]:
search_query = input("Enter the question")

results = RAG.search(search_query, k=1)
print(results)

[{'doc_id': 0, 'page_num': 2, 'score': 22.0, 'metadata': {}, 'base64': None}]


In [6]:
images = convert_from_path("/home/dataset1.pdf")
image_indexes = [pages['page_num']-1 for pages in results]

In [ ]:
# Qwen model
%pip install flash-attn qwen_vl_utils

In [8]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, AutoModelForImageTextToText, Qwen2VLForConditionalGeneration, AutoConfig
from qwen_vl_utils import process_vision_info

# config = AutoConfig.from_pretrained(
#     "Qwen/Qwen2-VL-2B-Instruct",
#     trust_remote_code=True
# )
# config.attn_config = {
#     "attn_type": "flash_attention",
#     "attn_implementation": "flash_attention_2"
# }
model = Qwen2VLForConditionalGeneration.from_pretrained(
                                                        "Qwen/Qwen2-VL-2B-Instruct", 
                                                        torch_dtype="auto",  
                                                        device_map = "auto",
                                                        # attn_implementation="flash_attention_2"
                                                        # config = config
                                                        )
model.cuda().eval()

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct", trust_remote_code=True)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
def generate_template(image_indexes):
    content = []
    for i in image_indexes:
        content.append({"type":"image", "image":images[i]})
    return content

content = generate_template(image_indexes)
content.append({"type": "text", "text": search_query})

In [10]:


messages = [
    {
        "role": "user",
        "content": content
    }
]



text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


generated_ids = model.generate(**inputs, max_new_tokens=50)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['The reason for transferring Mr. Tan Ah Kow from ABC Hospital to XYZ Hospital was due to his stroke. The clinical impression was that he was manifesting behavioural and psychological symptoms secondary to dementia.']
